In [1]:
"""
Quantum Support Vector Machine (QSVM) for Quantum Phase Transitions (QPT)

The QSVM program is structured as following:

1- Obtain the parameters for the circuit used to computed the Kernel.
These parameters are used as Training Data for the SVM, and are obtained via VQE 
from another program and defines the Ground States (GS) 
of the Ising Hamiltonian (on the ansatz circuit EfficientSU2).

2- Computed the Kernel using the Fidelity.

3- Train the Support Vector Machines using the sklearn package.

4- Find the Jc by minimizing the distance function. In this step the VQE is
implemented in order to find the GS for a generic J, and evaluate his distance
from the hyperplane find by the trained SVM.

"""

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library import EfficientSU2
from qiskit import Aer
from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import X,Z,I
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info.operators import Operator
from sklearn import svm

backend_sim = Aer.get_backend('qasm_simulator')
backend_sim.set_options(precision='single')

backend_sv = AerSimulator(method='statevector')
backend_sv.set_options(precision='single')

#Parameters for the Ansatz circuit (number of times of the CX block)
reps = 1

quantum_instance = QuantumInstance(backend=backend_sv, seed_transpiler=1)
optimizer = COBYLA(tol=0.01)
intermediate_info = dict()

In [ ]:
def circuit(N):
    """Define the circuit for the Quantum Kernel"""

    qr = QuantumRegister(N)
    qc = QuantumCircuit(qr)
    GS1 = EfficientSU2(N, su2_gates=['ry'], reps=reps, entanglement='linear', insert_barriers=True, parameter_prefix='p1')
    GS2 = EfficientSU2(N, su2_gates=['ry'], reps=reps, entanglement='linear', insert_barriers=True, parameter_prefix='p2')
    
    qc.append(GS1,qr)
    qc.append(GS2.inverse(),qr)
    qc.measure_all(add_bits=True)

    return qc



def bind_circuit(c, pi, pj):
    """bind the parameters to the circuit defined for the Quantum Kernel"""

    #p_{i} parameters from the GS_{i} and p_{j} parameters from the GS_{j}
    p = np.append(pi, pj)
    bind_dict = {c.parameters[i]: p[i] for i in range(c.num_parameters)}
    qc = c.bind_parameters(bind_dict)

    job_sim = backend_sim.run(transpile(qc, backend_sim), shots=nshots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)

    return counts



def Qkernel(p1, p2):
    """Quantum Kernel algorithm using the Fidelity"""

    c = circuit(N)

    x = len(p1)
    y = len(p2)
    K = np.empty((x, y), dtype=np.float64, order='C')

    for i in range(x):
        for j in range(i, y):
            
            counts = bind_circuit(c, p1[i], p2[j])
            if i == j:
                K[i,j] = 1
            else:
                try:
                    K[i, j] = counts[N*'0'] / nshots
                    K[j, i] = counts[N*'0'] / nshots
                except KeyError:
                    #if counts[N*'0'] is zero, then the relative Key (N*'0') isn't present in the dictionary (counts)
                    K[i, j] = 0.
                    K[j, i] = 0.
    K = np.sqrt(K)

    return K



def Qgram(p1, p2):
    """Quantum Kernel algorithm using the Fidelity"""

    c = circuit(N)

    x = len(p2)
    G = np.empty(shape=x, dtype=np.float64, order='C')

    for i in range(x):
        counts = bind_circuit(c, p1, p2[i])
        try:
            G[i] = counts[N*'0'] / nshots
        except KeyError:
            #if counts[N*'0'] is zero, then the relative Key (N*'0') isn't present in the dictionary (counts)
            G[i] = 0.
                
    G = np.sqrt(G)
    G = G.reshape(1, -1)

    return G



def Qdistance(J):
    """distance function according to the paper.
    Pass only 'J'. The relative set of parameters 'p' will be
    compute by the VQE (callable via GS()) inside this function.
    The parameters 'p' will be used to evaluate the 'distance' from
    the SVM iperplane.
    """
    c = circuit(N)
    p = GS(N, J)
    d = 0
    b = 0
    for i in range(M):
        for j in range(i, M):
            b += al[i]*ys[i]*K[clf.support_[i], clf.support_[j]] - ys[j]
            
    b = b/M
    d += b
    for j in range(M):
        counts = bind_circuit(c, p, sv[j])
        try:
            d += al[j]*ys[j]*np.sqrt(counts[N*'0'] / nshots)
        except KeyError:
            d += 0

    return float(d)



def Qdistance2(J):
    """Distance function using the decision_function method from sklearn"""

    p = GS(N, J)
    G = Qgram(p, p_train)
    d = clf.decision_function(G)

    return float(d)



def Hamiltonian(N, J):
    """The fans favorite Hamiltonian"""

    H = -J*(X^(I^N-2)^X) + (I^(N-1)^Z)
    for i in range(N-1):
        H += -J*((I^i)^(X^X)^(I^(N-2-i))) + ((I^i)^(Z)^(I^(N-i-1)))

    return H



def callback(nfev, parameters, energy, stddev):
    """Callback function required from the VQE to obtain
    the intermediate info, like the set of parameters of the
    optimized ansatz circuit.
    """

    intermediate_info['parameters'] = parameters
    intermediate_info['energy'] = energy

    

def GS(N, J):
    """Call the VQE to find the parameters that will defines the GS (on the Ansatz circuit)"""

    H = Hamiltonian(N, J)
    ansatz = EfficientSU2(N, su2_gates=['ry'], reps=reps, entanglement='linear', insert_barriers=True)
    #np.random.seed(16)
    np.random.seed(80)
    initial_point = np.random.random(N*(reps+1))

    local_vqe = VQE(ansatz=ansatz,
                    optimizer=optimizer,
                    initial_point=initial_point,
                    quantum_instance=quantum_instance,
                    callback=callback)

    local_result = local_vqe.compute_minimum_eigenvalue(H)
    par = intermediate_info['parameters']

    return par



def J_critical():
    """Minimize the distance function in order to find Jc"""

    results = root_scalar(Qdistance2, bracket=(sv_J[0], sv_J[-1]))

    if not results.converged:
        raise Exception(results.flag)
    else:
        return results.root



def save_JcVSN(delta):
    """Hope you have enough money to buy some indulgences and be saved.
    Save J_[c] VS N in columns'
    """

    f = open(f'results/noiseless/JcVSN_{delta}.txt', 'a')

    f.write(str(round(Jc, 3)) + ', ')
    f.write(str(N) + ', ')

    f.write('\n')
    f.close()


In [ ]:
def distance_vet(p):
    """Distance function using the method decision_function of sklearn"""
    p = p.reshape(1, -1)
    G = Qgram(p, p_train)
    d = clf.decision_function(G)

    return float(d)



def save_dVSJ(file):
    """save dVSJN.txt file. 
    If file = True, the GS parameters of each J will be read from the file
    tuttoN.txt, else they will be computed via VQE.
    DeltaJ = [0.25, 1.75] according to the paper
    """

    f = open(f'results/noiseless/distVSJ{N}.txt','w')
    f2 = open(f'results/noiseless/distVSJ{N}_SV.txt','w')
    f2.write(f'#Format: J VS d in columns, {2*M} data. \n')

    if file:
        a = np.genfromtxt(f'dataset/tutto{N}.txt', delimiter=',')
        DeltaJ = a[:,0]
        par = a[:,2:-1]
        f.write(f'#Format: J VS d in columns, {2*len(DeltaJ)} data. \n')

        for i in range(len(DeltaJ)):
            dJ = distance_vet(par[i])
            f.write(str(DeltaJ[i]) + ', ')
            f.write(str(round(dJ, 4)) + '\n')

    else:
        DeltaJ = [0.25 + 0.015*x for x in range(101)]
        f.write(f'#Format: J VS d in columns, {2*len(DeltaJ)} data. \n')

        for J_d in DeltaJ:
            dJ = distance2(J_d)
            f.write(str(J_d) + ', ')
            f.write(str(round(dJ, 4)) + ', \n')

    for i in range(M):
        sv_d = distance_vet(sv[i])
        f2.write(str(sv_J[i]) + ', ')
        f2.write(str(round(sv_d, 4)) + '\n')

    f.close()

In [2]:
def HarryPlotter_dvsJ(N):
    """It's magick as Fuck, it can even plot your soul (but is not a psychotherapist)"""

    a = np.genfromtxt(f'results/noiseless/distVSJ{N}.txt', delimiter=',')
    a2 = np.genfromtxt(f'results/noiseless/distVSJ{N}_SV.txt', delimiter=',')
    J = a[:, 0]
    d = a[:, 1]
    J_sv = a2[:, 0]
    d_sv = a2[:, 1]

    fig, ax = plt.subplots(figsize=(5,4), layout='constrained')

    ax.set_xlabel("J", size=15)
    ax.set_ylabel("distance", size=15)
    ax.set_title(f'd V.s. J for N = {N}')
    ax.text((Jc + 0.1), -0.2, r'$J_{c} =$' + str(round(Jc, 3)))

    ax.text(J_train[len(J_train)//2], 0.1, r'$\delta J_{1}$', c='dimgray', size=15)
    ax.text(J_train[0], 0.1, r'$\delta J_{2}$', c='dimgray', size=15)
    ax.axvspan(J_train[0], J_train[len(J_train)//2-1], alpha=0.5, color='gainsboro')
    ax.axvspan(J_train[len(J_train)//2], J_train[len(J_train)-1], alpha=0.5, color='lightgrey')

    ax.plot(J, d, c='midnightblue', label=f'd N={N}')
    ax.plot(J, np.zeros(len(J)), c='dimgray', linestyle=':')
    ax.scatter(np.round(sv_J, 1), np.round(d_sv), facecolor='None', edgecolor='midnightblue', s=40, label=f'SV N={N}')
    plt.legend()
    #plt.savefig(f'results/noiseless/dVSJ{N}.pdf', bbox_inches='tight')


In [ ]:
def HarryPlotter_JcVSN():
    """Harry Plotter and the Quantum Phase Transitions"""

    c = [None, 'purple', 'rebeccapurple', 'coral']
    fig, ax = plt.subplots(figsize=(5,4), layout='constrained')

    ax.set_ylabel(r"$J_{c}$", size=15)
    ax.set_xlabel("N", size=15)

    N = [i for i in range(4, 13)]
    ax.plot(N, np.ones(len(N)), c='dimgray', linestyle=':')

    for i in range(1, 4):
        file = np.genfromtxt(f'results/noiseless/JcVSN_{i}.txt', delimiter=',')
        #file2 = np.genfromtxt(f'results/noiseless/JcVSN_anal_{i}.txt', delimiter=',')
        J = file[:, 0]
        #J2 = file2[:, 0]

        ax.plot(N, J, linestyle='--', c=f'{c[i]}', linewidth=1)
        ax.scatter(N, J, s=50, c=f'{c[i]}', label=rf"$\delta_{i}$")
        #ax.plot(N, J2, linestyle='--', c=f'{c[i]}', linewidth=1)
        #ax.scatter(N, J2, s=50, facecolor='none', edgecolors=f'{c[i]}')

    plt.legend()
    #plt.savefig('results/noiseless/JcVSN.pdf', bbox_inches='tight')


In [ ]:
def plot_Kmatrix(K):
    """Plot the Kernel"""

    fig = plt.figure(layout='constrained')
    ax = fig.add_subplot()
    cax = ax.matshow(K, cmap=plt.cm.get_cmap('cividis', ), interpolation='nearest')
    fig.colorbar(cax)

    ax.set_title('Kernel', size=20)
    #plt.savefig(f'results/noiseless/Kernel{N}.pdf', bbox_inches='tight')

In [ ]:
Nmin = 6
Nmax = 7
nshots = 1024
for N in range(Nmin, Nmax, 2):

    #obtaining the data parameters for the circuit (computed by the VQE in another program)
    file = np.genfromtxt(f'dataset/tutto{N}.txt', delimiter=',')
    #delta = 1 --> (0.4, 0.5) U (1.4, 1.5)
    #delta = 2 --> (0.6, 0.7) U (1.1, 1.2)
    #delta = 3 --> (0.7, 0.8) U (1.4, 1.5)
    delta = 00
    J_train = file[:, 0]
    p_train = file[:, a:-1]
    #The following array is necessary for the data classification
    y_train = [1 if x > 1 else -1 for x in J_train]
    
    #compute the Kernel
    K = Qkernel(p_train, p_train)
    #Train the SVM for classification
    clf = svm.SVC(kernel='precomputed', cache_size=1000)
    clf.fit(K, y_train)

    #support vectors info
    sv = p_train[clf.support_]
    sv_J = J_train[clf.support_]
    ys = np.reshape(y_train, (len(y_train), 1))[clf.support_]
    al = np.abs(clf.dual_coef_[0])
    M  = len(sv)

    #find J_{c}
    Jc = J_critical()
    #save_JcVSN(delta)
    print(Jc)
    

In [ ]:
plot_Kmatrix(K)

In [ ]:
#Save and plot distance VS J
#save_dVSJ(False)
HarryPlotter_dvsJ(N)

In [ ]:
HarryPlotter_JcVSN()